## Setup

In [1]:
#import needed packages
import os
import pandas as pd
import requests as re
import googlemaps

from IPython.display import JSON

In [2]:
#import custom function python
%run -i ../notebooks/functions/functions.py

## JSON to DataFrame

See MiniProject 4 Google Doc for more details

In [3]:
#roots of the different datasets
#Housing
construction_root = 'https://data.cityofnewyork.us/resource/hg8x-zxpr.json'
hbd_root = 'https://data.cityofnewyork.us/resource/kj4p-ruqc.json'

#Wellbeing
facilities_root = 'https://data.cityofnewyork.us/resource/ji82-xba5.json'
places_root = 'https://data.cityofnewyork.us/resource/ssdk-4qjy.json'
parks_root = 'https://data.cityofnewyork.us/resource/enfh-gkve.json'
hotels_root = 'https://data.cityofnewyork.us/resource/tjus-cn27.json'
library_root = 'https://data.cityofnewyork.us/resource/feuq-due4.json'

#Business
business_root = 'https://data.cityofnewyork.us/resource/w7w3-xahh.json'

### Libraries, Parks & Hotels

In [4]:
#-- API calls-- 
#no filter required
library_JSON = API(library_root, None, None)
#only pull active parks
parks_JSON = API(parks_root, '?retired=false&$limit=3000', None)
#only pull hotels listed from tax year 2021
hotels_JSON = API(hotels_root, '?taxyear=2021&$limit=6000', None)

#### Libraries

In [5]:
#pull out desired feats from library
desired_feat = ['the_geom']
library_geom = JSON_to_DF(library_JSON, desired_feat)

#convert geometry point data to coordinates
library_df = coord_from_geom(library_geom)
print(library_df.shape)
library_df.head()

(216, 2)


,latitude,longitude
0,40.802980,-73.953531
1,40.803018,-73.934848
2,40.760807,-73.977363
3,40.762186,-73.969382
4,40.764915,-73.959550


#### Parks

In [6]:
#pull out desired feats from parks
desired_feat = ['multipolygon', 'acres']
parks_street = JSON_to_DF(parks_JSON, desired_feat)
print(parks_street.shape)
parks_street.head()

(2025, 2)


,multipolygon,acres
0,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",21.10936646
1,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.89
2,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",5.739
3,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.001
4,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.032


In [7]:
#define empty lat, long lists
latitude = []
longitude = []

#for each row 
for row in range(parks_street.shape[0]):
    #pull out polygon coordinates
    polygon = parks_street.multipolygon[row]['coordinates'][0][0]
    #find the center
    center = centroid(polygon)
    #append to lat and long (coordinate order is swapped)
    #latitude
    latitude.append(center[1])
    #longitude
    longitude.append(center[0])

#add longitude and latiude as columns
parks_street['latitude'] = latitude
parks_street['longitude'] = longitude
#drop geometry column   
parks_df = parks_street.drop('multipolygon', axis = 1)
parks_df.head()

,acres,latitude,longitude
0,21.10936646,40.796328,-73.897520
1,0.89,40.682928,-73.930628
2,5.739,40.690340,-73.999386
3,0.001,40.730382,-73.874960
4,0.032,40.621921,-74.022569


In [8]:
#coerce dtype to be numeric
parks_df = parks_df.apply(pd.to_numeric, errors='coerce')
parks_df.dtypes

acres        float64
latitude     float64
longitude    float64
dtype: object

#### Hotels

In [9]:
#transform from JSON to DF
desired_feats = ['latitude', 'longitude']
hotels_df = JSON_to_DF(hotels_JSON, desired_feats)
#check
print(hotels_df.shape)
hotels_df.head()

(2731, 2)


,latitude,longitude
0,40.703235,-74.012421
1,40.702744,-74.012201
2,40.704025,-74.012638
3,40.704039,-74.012317
4,40.714812,-74.016153


In [10]:
#coerce dtype to be numeric
hotels_df = hotels_df.apply(pd.to_numeric, errors='coerce')

### Facilities

Bigger sets, want to pull just based on specific categories so we're getting a clearer picture

See `facilities_datadictionary.xlsx` for possible facility category, subcatory, and factype

#--- ORIGINAL SUBGROUPS --
#define subgroups we want
facility_subgroup = ['BUS DEPOTS AND TERMINALS', 'PARKING LOTS AND GARAGES', 
                     'MUSEUMS', 'STREETSCAPES, PLAZAS, AND MALLS', 
                     'NON-PUBLIC K-12 SCHOOLS', 'PUBLIC K-12 SCHOOLS', 'COLLEGES OR UNIVERSITIES', 
                    'HOSPITALS AND CLINICS'] 

In [11]:
#--- EXPERIMENT SUBGROUPS --
facility_subgroup = ['PUBLIC K-12 SCHOOLS', 'NON-PUBLIC K-12 SCHOOLS',
                     'HOSPITALS AND CLINICS', 'DAY CARE'] 

In [12]:
#-- API call--

#make dict to store JSON returns
facility_JSONs = {}

#loop through each subgroup and call JSON
for subgroup in facility_subgroup:
    #define endpoint with higher limit
    endpoint = facilities_root + f"?facsubgrp={subgroup}&$limit=3000"
    #GET
    response = re.get(endpoint, 
                      data={'app_token': os.environ['NYC_TOKEN']})
   
    #return status code and results
    status_code, facility_JSONs[subgroup] = response.status_code, response.json()

In [13]:
#number of return values for each JSON
for JSON in facility_JSONs:
    print(JSON, len(facility_JSONs[JSON]))

PUBLIC K-12 SCHOOLS 1515
NON-PUBLIC K-12 SCHOOLS 1113
HOSPITALS AND CLINICS 1191
DAY CARE 2264


In [14]:
#name lat, long columns
facility_features = ['latitude', 'longitude']
#create empty dict
facility_df = {}
#for every JSON, translate to pandas DF and store into dict under its name
for JSON in facility_JSONs:
    facility_df[JSON] = JSON_to_DF(facility_JSONs[JSON], facility_features)

In [15]:
#test print a facility dataframe
facility_df[facility_subgroup[0]].head()

,latitude,longitude
0,40.63660846100,-74.13247581100
1,40.88822009500,-73.85266908930
2,40.71828398100,-74.01096910700
3,40.59793367110,-74.07013452940
4,40.82143759600,-73.85589683460


In [16]:
#concatinate all facilities into one dataframe, using title as factype column
facility_all = pd.concat(facility_df).reset_index().drop('level_1', axis = 1)
facility_all = facility_all.rename(columns={'level_0': 'factype'})
#check
facility_all.head()

,factype,latitude,longitude
0,PUBLIC K-12 SCHOOLS,40.63660846100,-74.13247581100
1,PUBLIC K-12 SCHOOLS,40.88822009500,-73.85266908930
2,PUBLIC K-12 SCHOOLS,40.71828398100,-74.01096910700
3,PUBLIC K-12 SCHOOLS,40.59793367110,-74.07013452940
4,PUBLIC K-12 SCHOOLS,40.82143759600,-73.85589683460


In [17]:
#only include datapoints with lat/long values
facility_clean = facility_all.loc[facility_all['latitude'] != 0]

In [18]:
#coerce dtype of lat and long to be numeric, leave factype as object
facility_clean[['latitude', 'longitude']] = facility_clean[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')
facility_clean.dtypes

factype       object
latitude     float64
longitude    float64
dtype: object

### To csv

In [19]:
#save final dataframes to csv
library_df.to_csv('../processed_data/wellbeing/libraries.csv', index=False)
parks_df.to_csv('../processed_data/wellbeing/parks.csv', index=False)
hotels_df.to_csv('../processed_data/wellbeing/hotels.csv', index=False)
facility_clean.to_csv('../processed_data/wellbeing/facilities.csv', index=False)

In [20]:
facility_clean.factype.unique()

array(['PUBLIC K-12 SCHOOLS', 'NON-PUBLIC K-12 SCHOOLS',
       'HOSPITALS AND CLINICS', 'DAY CARE'], dtype=object)

### Housing

#### House construction

In [21]:
#-- API call -- 

#call housing data within range
housing_start_JSON = range_SODA(construction_root, 'project_start_date', ['2021-01-01T00:00:00', '2021-12-30T00:00:00'])

In [22]:
#id desired features
desired_feat = ['total_units', 'latitude', 'longitude']
#convert house start to dataframe
house_start_df = JSON_to_DF(housing_start_JSON, desired_feat)
print(house_start_df.shape)

(405, 3)


In [23]:
#coerce dtype to be numeric
house_start_df = house_start_df.apply(pd.to_numeric, errors='coerce')
house_start_df.head()
house_start_df.to_csv('../processed_data/buildings/house_start.csv', index=False)

#### HPD buildings

In [24]:
#-- API call -- 
#call housing data within range
hbd_JSON = API(hbd_root,'?recordstatus=Active&lifecycle=Building&$limit=50000',
              None)

In [25]:
#translate hbd JSON to dataframe
desired_feat = ['zip', 'legalstories']
hbd_df = JSON_to_DF(hbd_JSON, desired_feat)

In [26]:
#check
hbd_df.head()

,zip,legalstories
0,10466,1
1,11216,3
2,10314,2
3,11375,2
4,11226,2


*Note*: the next part is commented out due to the generation of data from google API costing money. We can't rerun this part but we did get the data necessary out of it. See `building_count.csv` and `building_stories.csv`

In [27]:
#gmaps = googlemaps.Client(key=os.environ['TEMP_GOOGLE'])

In [28]:
#lat = []
#long = []

#-- DO NOT RUN -- 

#for zip_code in hbd_df.zip:

    #geocode_result = gmaps.geocode(f'{zip_code}, NY')
    #try:
        #lat.append(geocode_result[0]['geometry']['location']['lat'])
        #long.append(geocode_result[0]['geometry']['location']['lng'])
    #except:
        #lat.append('NaN')
        #long.append('NaN')

In [29]:
#append lat and long
#hbd_df['latitude'] = lat
#hbd_df['longitude'] = long

#drop zip codes
#hbd_df = hbd_df.drop('zip', axis=1)
#hbd_df = hbd_df.apply(pd.to_numeric, errors='coerce')

In [30]:
#hbd_df.info()

In [31]:
#NTA = pd.read_csv('../Mid-Term/Mid-Term-Project/processed_data/geocoded_population.csv').loc[:, ['latitude', 'longitude']]
#print(NTA.shape)
#NTA.head()

In [32]:
#bin within the facilities
#building_count = bin_data(hbd_df,'count', 'buildings', 70)
#building_level = bin_data(hbd_df, 'mean', 'buildings', 1)

In [33]:
#building_level.head()

In [34]:
#building_count.loc[building_count['buildings'] != 0].describe()

In [35]:
# -- DO NOT OVERWRITE SAVED BUILDING DATA --

#building_count.to_csv('../Mid-term/Mid-Term-Project/processed_data/building_count.csv', index=False)
#building_level.to_csv('../Mid-term/Mid-Term-Project/processed_data/building_stories.csv', index=False)